In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [49]:
import pandas as pd
import numpy as np
# import joblib
from datetime import datetime
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
# from imblearn.over_sampling import SMOTE
# from collections import Counter
# from imblearn.under_sampling import RandomUnderSampler

In [50]:
import kagglehub

# Download latest version
# path = kagglehub.dataset_download("kaiyoo88/fake-real-estate")

path = "/kaggle/input/open123123123"
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/open123123123


In [53]:
# 데이터 로드
train = pd.read_csv(f'{path}/train.csv')

In [52]:
# Feature & Target 설정
x = train.drop(['ID', '허위매물여부'], axis=1)
y = train['허위매물여부']

In [54]:
# 1) 결측치 처리: KNN Imputer 사용 (더 정밀한 방식)
knn_imputer = KNNImputer(n_neighbors=5)  # K=5로 설정하여 결측치 예측
columns_fill_knn = ['해당층', '총층', '전용면적', '방수', '욕실수', '총주차대수']
x[columns_fill_knn] = knn_imputer.fit_transform(x[columns_fill_knn])

In [55]:
## 2) Feature Engineering 추가

def create_new_features(x):
    x = x.copy() 
    
    # 1 단위면적당 가격 (㎡당 가격)
    x['단위면적당가격'] = x['보증금'] / x['전용면적']
    x['단위면적당가격'].fillna(x['단위면적당가격'].median())
    
    # 2 보증금 대비 월세 비율
    x['보증금_월세비율'] = x['보증금'] / (x['월세'] + 1)
    x['보증금_월세비율'].fillna(x['보증금_월세비율'].median())
    
    # 3 층수 비율 (해당층 / 총층)
    x['층수_비율'] = x['해당층'] / x['총층']
    x['층수_비율'].fillna(x['층수_비율'].median())
    
    # 4 게재일 관련 Feature
    x['게재일'] = pd.to_datetime(x['게재일'])
    # x['게재요일'] = x['게재일'].dt.weekday
    x['게재일_연도'] = x['게재일'].dt.year  # 연도
    x['게재일_월'] = x['게재일'].dt.month  # 월
    x['게재일_요일'] = x['게재일'].dt.weekday  # 요일 (0=월요일, 6=일요일)
    x['게재일_경과일'] = (datetime.today() - x['게재일']).dt.days
    # '게재일' 원본 컬럼 제거 (불필요)
    x = x.drop(columns=['게재일'])
    
    # 5 방향 그룹화
    direction_map = {'동향': '동', '서향': '서', '남향': '남', '북향': '북', '남동향': '남', '북동향': '북'}
    x['방향_그룹'] = x['방향'].map(direction_map)
    
    # 6 이상 가격 탐지 Feature
    unit_price_mean = x['단위면적당가격'].mean()
    unit_price_std = x['단위면적당가격'].std()
    x['가격_이상치'] = ((x['단위면적당가격'] - unit_price_mean) / unit_price_std).abs()
    
    # 7 주차 가능 여부 수치 변환
    x['주차가능여부'] = x['주차가능여부'].map({'가능': 1, '불가능': 0})
    
    # 8 월세 + 관리비 총 비용
    x['월세_총비용'] = x['월세'] + x['관리비']
    x['월세_총비용'].fillna(x['월세_총비용'].median())
    
    # 9 관리비 비율 (관리비 / 월세)
    x['관리비_비율'] = x['관리비'] / (x['월세'] + 1)
    x['관리비_비율'].fillna(x['관리비_비율'].median())
    
    # 10 방수 밀집도 (방수 / 전용면적) & 욕실 밀집도 (욕실수 / 전용면적)
    x['방수_밀집도'] = x['방수'] / (x['전용면적'] + 1)
    x['욕실_밀집도'] = x['욕실수'] / (x['전용면적'] + 1)
    
    # 11. 플랫폼별 평균 보증금 / 월세 차이
    플랫폼_보증금평균 = x.groupby('제공플랫폼')['보증금'].transform('mean')
    플랫폼_월세평균 = x.groupby('제공플랫폼')['월세'].transform('mean')
    
    x['제공플랫폼_보증금차이'] = x['보증금'] - 플랫폼_보증금평균
    x['제공플랫폼_월세차이'] = x['월세'] - 플랫폼_월세평균
    
    return x

x = create_new_features(x)

In [59]:
# 5) Train / Validation 분할 (Stratified 방식)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [60]:
def add_noise(df, noise_level=0.02): #0.05
    numeric_cols = df.select_dtypes(include=[np.number]).columns  # 숫자형 컬럼만 선택
    df[numeric_cols] = df[numeric_cols] * (1 + noise_level * np.random.randn(*df[numeric_cols].shape))
    return df

print("Before Noise Injection:", len(x_train))
x_train = add_noise(x_train)
print("After Noise Injection:", len(x_train))

Before Noise Injection: 1961
After Noise Injection: 1961


In [10]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# cat_features 정의
cat_features = ['매물확인방식', '중개사무소', '제공플랫폼', '방향', '방향_그룹']

# 🛠 범주형 컬럼 확인 및 누락 처리
for col in cat_features:
    if col not in x.columns:
        print(f"'{col}' 컬럼이 x에 존재하지 않습니다. 기본값 '미확인'으로 추가합니다.")
        x[col] = '미확인'
    x[col] = x[col].astype(str)

for col in cat_features:
    if col not in x_val.columns:
        print(f"'{col}' 컬럼이 x_val에 존재하지 않습니다. 기본값 '미확인'으로 추가합니다.")
        x_val[col] = '미확인'
    x_val[col] = x_val[col].astype(str)

# CatBoost 모델 초기화
cat_model = CatBoostClassifier(
    cat_features=cat_features,
    auto_class_weights="Balanced",  # 클래스 비율 자동 균형화
    verbose=0
)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'iterations': [500, 1000],    # 반복 횟수
    'depth': [6, 8, 10],          # 트리 깊이
    'learning_rate': [0.03, 0.07], # 학습률
    'l2_leaf_reg': [3, 5, 7],      # 정규화 파라미터
}

# Stratified K-Fold 적용
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV 실행
grid_search = GridSearchCV(
    estimator=cat_model,
    param_grid=param_grid,
    cv=skf,                   # ✅ Stratified K-Fold 적용
    scoring='f1_macro',       # ✅ Macro F1 Score 기준
    n_jobs=-1,                # ✅ 병렬 처리
    verbose=1                 # ✅ 진행상황 출력
)

# Grid Search 실행
grid_search.fit(x, y)

# 최적 하이퍼파라미터 및 성능 출력
print("Best parameters:", grid_search.best_params_)
print("Best Macro F1-score:", grid_search.best_score_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'depth': 10, 'iterations': 500, 'l2_leaf_reg': 7, 'learning_rate': 0.07}
Best Macro F1-score: 0.9260898945130162


In [46]:
print(cat_features)

['매물확인방식', '중개사무소', '제공플랫폼', '방향', '방향_그룹']


In [88]:
# 🛠 범주형 컬럼 확인 및 누락 처리
for col in cat_features:
    if col not in x.columns:
        print(f"'{col}' 컬럼이 x에 존재하지 않습니다. 기본값 '미확인'으로 추가합니다.")
        x[col] = '미확인'
    x[col] = x[col].astype(str)

for col in cat_features:
    if col not in x_val.columns:
        print(f"'{col}' 컬럼이 x_val에 존재하지 않습니다. 기본값 '미확인'으로 추가합니다.")
        x_val[col] = '미확인'
    x_val[col] = x_val[col].astype(str)

# NaN 값을 '미확인'으로 대체
for col in cat_features:
    if col in x.columns:
        x[col].fillna('미확인', inplace=True)
    if col in x_val.columns:
        x_val[col].fillna('미확인', inplace=True)

# CatBoost 모델 초기화
cat_model = CatBoostClassifier(
    cat_features=cat_features,
    auto_class_weights="Balanced",  # 클래스 비율 자동 균형화
    verbose=0
)

# (이후의 코드 생략)


<ipython-input-88-2595c0b63b05>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x[col].fillna('미확인', inplace=True)
<ipython-input-88-2595c0b63b05>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method

In [90]:
# Test 데이터 로드
test = pd.read_csv(f'{path}/test.csv')

In [91]:
# Test 결측값 대체
test[columns_fill_knn] = knn_imputer.transform(test[columns_fill_knn])

In [92]:
# Test 결측값 대체
test[columns_fill_knn] = knn_imputer.transform(test[columns_fill_knn])

In [93]:
# def create_new_features(x): 
#     # 1 단위면적당 가격 (㎡당 가격)
#     x['단위면적당가격'] = x['보증금'] / x['전용면적']
#     x['단위면적당가격'].fillna(x['단위면적당가격'].median(), inplace=True)
    
#     # 2 보증금 대비 월세 비율
#     x['보증금_월세비율'] = x['보증금'] / (x['월세'] + 1)
#     x['보증금_월세비율'].fillna(x['보증금_월세비율'].median(), inplace=True)
    
#     # 3 층수 비율 (해당층 / 총층)
#     x['층수_비율'] = x['해당층'] / x['총층']
#     x['층수_비율'].fillna(x['층수_비율'].median(), inplace=True)
    
#     # 4 게재일 관련 Feature
#     x['게재일'] = pd.to_datetime(x['게재일'])
#     x['게재요일'] = x['게재일'].dt.weekday
#     x['게재일_경과일'] = (datetime(2025, 1, 20) - x['게재일']).dt.days
    
#     # 5 방향 그룹화
#     direction_map = {'동향': '동', '서향': '서', '남향': '남', '북향': '북', '남동향': '남', '북동향': '북'}
#     x['방향_그룹'] = x['방향'].map(direction_map)
    
#     # 6 이상 가격 탐지 Feature
#     unit_price_mean = x['단위면적당가격'].mean()
#     unit_price_std = x['단위면적당가격'].std()
#     x['가격_이상치'] = ((x['단위면적당가격'] - unit_price_mean) / unit_price_std).abs()
    
#     # 7 주차 가능 여부 수치 변환
#     x['주차가능여부'] = x['주차가능여부'].map({'가능': 1, '불가능': 0})
    
#     # 8 월세 + 관리비 총 비용
#     x['월세_총비용'] = x['월세'] + x['관리비']
#     x['월세_총비용'].fillna(x['월세_총비용'].median(), inplace=True)
    
#     # 9 관리비 비율 (관리비 / 월세)
#     x['관리비_비율'] = x['관리비'] / (x['월세'] + 1)
#     x['관리비_비율'].fillna(x['관리비_비율'].median(), inplace=True)
    
#     # 10 방수 밀집도 (방수 / 전용면적) & 욕실 밀집도 (욕실수 / 전용면적)
#     x['방수_밀집도'] = x['방수'] / (x['전용면적'] + 1)
#     x['욕실_밀집도'] = x['욕실수'] / (x['전용면적'] + 1)
    
#     # 11 플랫폼별 평균 보증금 / 월세 차이
#     플랫폼_보증금평균 = train.groupby('제공플랫폼')['보증금'].mean()
#     플랫폼_월세평균 = train.groupby('제공플랫폼')['월세'].mean()
    
#     x['제공플랫폼_보증금차이'] = x['보증금'] - x['제공플랫폼'].map(플랫폼_보증금평균)
#     x['제공플랫폼_월세차이'] = x['월세'] - x['제공플랫폼'].map(플랫폼_월세평균)
#     return x

# test = create_new_features(test)
# test.head()

test = create_new_features(test)

In [94]:
for col in cat_features:
    test[col] = test[col].astype(str)

In [95]:
test.drop(columns=['ID'], inplace=True)

In [96]:
pred = pd.Series(cat_model.predict(test)) #best_model

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

In [98]:
# CatBoost 모델 초기화
cat_model = CatBoostClassifier(
    cat_features=cat_features,
    auto_class_weights="Balanced",  # 클래스 비율 자동 균형화
    verbose=0
)

# 모델 학습
cat_model.fit(x, y)  # x는 학습 데이터, y는 레이블

In [99]:
# 테스트 데이터에 대한 예측
pred = pd.Series(cat_model.predict(test))  # test는 테스트 데이터


In [100]:
# 필요한 라이브러리 임포트
import pandas as pd
from catboost import CatBoostClassifier

# 데이터 준비 (x, y, test, cat_features 정의)

# NaN 값 처리 및 범주형 특성 변환 (이전 단계에서 설명한 대로)

# CatBoost 모델 초기화
cat_model = CatBoostClassifier(
    cat_features=cat_features,
    auto_class_weights="Balanced",  # 클래스 비율 자동 균형화
    verbose=0
)

# 모델 학습
cat_model.fit(x, y)  # x는 학습 데이터, y는 레이블

# 테스트 데이터에 대한 예측
pred = pd.Series(cat_model.predict(test))  # test는 테스트 데이터


In [102]:
pred = pd.Series(cat_model.predict(test)) #best_model

In [103]:
print('1:', pred.sum(), '| ratio:', (pred.sum()/len(pred)*100))

1: 74 | ratio: 12.071778140293638


In [104]:
submit = pd.read_csv(f'{path}/sample_submission.csv')

In [105]:
submit['허위매물여부'] = pred # 우리의 예측 넣는다
submit.head()

,ID,허위매물여부
0,TEST_000,0
1,TEST_001,0
2,TEST_002,1
3,TEST_003,0
4,TEST_004,0


In [106]:
submit.to_csv('./baseline_submission_220908.csv',index=False)